In [1]:
!unzip /content/io-t-network-attack-detection.zip

Archive:  /content/io-t-network-attack-detection.zip
  inflating: Sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [2]:
!pip -q install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 24.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.0/430.0 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
sample_submission = pd.read_csv('Sample_submission.csv')

In [4]:
print("Train dataset shape:", train_df.shape)
print("Test dataset shape:", test_df.shape)
print("Sample submission shape:", sample_submission.shape)

Train dataset shape: (61558, 85)
Test dataset shape: (61559, 85)
Sample submission shape: (61559, 2)


In [5]:
print(train_df.head())

   id  id.orig_p  id.resp_p proto service  flow_duration  fwd_pkts_tot  \
0   0      43296         53   udp     dns       0.020938             2   
1   1      33859         21   tcp     NaN       0.000004             1   
2   2      64606         21   tcp     NaN       0.000004             1   
3   3      54357        137   udp     dns       0.000000             1   
4   4      51036       1100   tcp     NaN       0.000006             1   

   bwd_pkts_tot  fwd_data_pkts_tot  bwd_data_pkts_tot  ...  active.std  \
0             2                  2                  2  ...         0.0   
1             1                  1                  0  ...         0.0   
2             1                  1                  0  ...         0.0   
3             0                  1                  0  ...         0.0   
4             1                  0                  0  ...         0.0   

   idle.min  idle.max  idle.tot  idle.avg  idle.std  fwd_init_window_size  \
0       0.0       0.0       0.0  

In [6]:
print("\nMissing values in train data:")
print(train_df.isnull().sum())
print("\nMissing values in test data:")
print(test_df.isnull().sum())


Missing values in train data:
id                          0
id.orig_p                   0
id.resp_p                   0
proto                       0
service                 51432
                        ...  
idle.std                    0
fwd_init_window_size        0
bwd_init_window_size        0
fwd_last_window_size        0
Attack_type                 0
Length: 85, dtype: int64

Missing values in test data:
id                          0
id.orig_p                   0
id.resp_p                   0
proto                       0
service                 51429
                        ...  
idle.std                    0
fwd_init_window_size        0
bwd_init_window_size        0
fwd_last_window_size        0
Attack_type             61559
Length: 85, dtype: int64


In [7]:
print("\nData types in train data:")
print(train_df.dtypes)
print("\nStatistical summary of train data:")
print(train_df.describe())


Data types in train data:
id                        int64
id.orig_p                 int64
id.resp_p                 int64
proto                    object
service                  object
                         ...   
idle.std                float64
fwd_init_window_size      int64
bwd_init_window_size      int64
fwd_last_window_size      int64
Attack_type              object
Length: 85, dtype: object

Statistical summary of train data:
                 id     id.orig_p     id.resp_p  flow_duration  fwd_pkts_tot  \
count  61558.000000  61558.000000  61558.000000   61558.000000  61558.000000   
mean   30778.500000  34698.806930   1028.740131       3.449925      2.166136   
std    17770.408272  19043.235091   5284.921627     114.932118     13.461114   
min        0.000000      0.000000      0.000000       0.000000      0.000000   
25%    15389.250000  17777.500000     21.000000       0.000001      1.000000   
50%    30778.500000  37353.000000     21.000000       0.000004      1.000000   

In [8]:
print("\nDistribution of attack types:")
attack_counts = train_df['Attack_type'].value_counts()
print(attack_counts)


Distribution of attack types:
Attack_type
DOS_SYN_Hping                 47329
Thing_Speak                    4054
ARP_poisioning                 3875
MQTT_Publish                   2073
NMAP_UDP_SCAN                  1295
NMAP_XMAS_TREE_SCAN            1005
NMAP_OS_DETECTION              1000
NMAP_TCP_scan                   501
DDOS_Slowloris                  267
Wipro_bulb                      126
Metasploit_Brute_Force_SSH       19
NMAP_FIN_SCAN                    14
Name: count, dtype: int64


In [9]:
plt.figure(figsize=(12, 6))
sns.countplot(y=train_df['Attack_type'], order=train_df['Attack_type'].value_counts().index)
plt.title('Distribution of Attack Types')
plt.tight_layout()
plt.savefig('attack_distribution.png')
plt.close()

In [10]:
categorical_features = train_df.select_dtypes(include=['object']).columns.tolist()
categorical_features.remove('Attack_type')  # Don't include the target variable
print(f"\nCategorical features: {categorical_features}")


Categorical features: ['proto', 'service']


In [11]:
for feature in categorical_features:
    le = LabelEncoder()
    combined_data = pd.concat([train_df[feature], test_df[feature]])
    le.fit(combined_data)

    train_df[feature] = le.transform(train_df[feature])
    test_df[feature] = le.transform(test_df[feature])

In [12]:
print("\nCardinality of categorical features:")
for feature in categorical_features:
    print(f"{feature}: {train_df[feature].nunique()} unique values")


Cardinality of categorical features:
proto: 3 unique values
service: 10 unique values


In [13]:
if 'frame.time' in train_df.columns:
    train_df['hour'] = pd.to_datetime(train_df['frame.time']).dt.hour
    test_df['hour'] = pd.to_datetime(test_df['frame.time']).dt.hour

In [14]:
if 'frame.len' in train_df.columns and 'data.len' in train_df.columns:
    train_df['packet_ratio'] = train_df['data.len'] / (train_df['frame.len'] + 1)  # +1 to avoid division by zero
    test_df['packet_ratio'] = test_df['data.len'] / (test_df['frame.len'] + 1)

In [15]:
label_column = 'Attack_type'
train_data = train_df.copy()
test_data = test_df.copy()

In [16]:
features = [col for col in train_data.columns if col != label_column]


In [17]:
print("\nTraining AutoGluon model...")
save_path = 'agModels-IoTAttackDetection'
predictor = TabularPredictor(label=label_column, path=save_path)
predictor.fit(
    train_data=train_data,
    time_limit=600,  # 1 hour time limit
    presets='best_quality'  # Use best quality preset for better performance
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.75 GB / 12.67 GB (84.8%)
Disk Space Avail:   190.34 GB / 235.68 GB (80.8%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacke


Training AutoGluon model...


	Running DyStack sub-fit in a ray process to avoid memory leakage. Enabling ray logging (enable_ray_logging=True). Specify `ds_args={'enable_ray_logging': False}` if you experience logging issues.
2025-03-15 12:30:52,471	INFO worker.py:1810 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
		Context path: "/content/agModels-IoTAttackDetection/ds_sub_fit/sub_fit_ho"
(_dystack pid=8303) Running DyStack sub-fit ...
(_dystack pid=8303) Beginning AutoGluon training ... Time limit = 145s
(_dystack pid=8303) AutoGluon will save models to "/content/agModels-IoTAttackDetection/ds_sub_fit/sub_fit_ho"
(_dystack pid=8303) Train Data Rows:    54718
(_dystack pid=8303) Train Data Columns: 84
(_dystack pid=8303) Label Column:       Attack_type
(_dystack pid=8303) Problem Type:       multiclass
(_dystack pid=8303) Preprocessing data ...
(_dystack pid=8303) Train Data Class Count: 12
(_dystack pid=8303) Using Feature Generators to preprocess the data ...
(_dystack pid=8303) 

In [18]:
print("\nEvaluating model performance...")
leaderboard = predictor.leaderboard()
print(leaderboard)


Evaluating model performance...
                    model  score_val eval_metric  pred_time_val    fit_time  \
0       LightGBMXT_BAG_L1   0.996296    accuracy       6.314547   80.714876   
1     WeightedEnsemble_L2   0.996296    accuracy       6.326919   82.099215   
2  NeuralNetFastAI_BAG_L1   0.994850    accuracy       1.215582  273.241106   
3   KNeighborsDist_BAG_L1   0.981107    accuracy      32.760483    0.163191   
4   KNeighborsUnif_BAG_L1   0.979531    accuracy      31.715187    0.219105   

   pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
0                6.314547          80.714876            1       True   
1                0.012372           1.384338            2       True   
2                1.215582         273.241106            1       True   
3               32.760483           0.163191            1       True   
4               31.715187           0.219105            1       True   

   fit_order  
0          4  
1          5  
2          3  

In [ ]:
# feature_importance = predictor.feature_importance(train_data)
# print("\nFeature importance:")
# print(feature_importance)

These features in provided data are not utilized by the predictor and will be ignored: ['bwd_URG_flag_count']
Computing feature importance via permutation shuffling for 83 features using 5000 rows with 5 shuffle sets...
/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")
/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only nee

In [19]:
print("\nMaking predictions on test data...")
test_predictions = predictor.predict(test_data)
test_probabilities = predictor.predict_proba(test_data)


Making predictions on test data...


In [20]:
submission = sample_submission.copy()
submission[label_column] = test_predictions
submission.to_csv('submission_1.csv', index=False)
print("Submission file created: submission.csv")

Submission file created: submission.csv
